In [197]:
import requests
from bs4 import BeautifulSoup 
from tqdm import tqdm
import re
import pandas as pd

In [ ]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}


In [279]:
def get_links(year, month:str, page):
    url ='http://www.arthub.co.kr/sub01/board033_list.htm?'
    seed = 'http://www.arthub.co.kr/sub01/'
    
    params = {
        'k_date':year,
        'k_month':month,
        'PHPSESSID':'e124fc854822909d9ec3618c1e6089f2',
        'page':page
        }
    
    resp = requests.get(url,params=params,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    aList = soup.select('a.sub01_list_title')
    return [(str(year)+month,a.get_text(),seed+a['href']) for a in aList]

In [229]:
get_links(2018,'08',1)

[('201808',
  '2018 후용아트폐허肺墟 :: Art Fair',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26567'),
 ('201808',
  '제리 테이퍼, 박형진 2人展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26718'),
 ('201808',
  '마주한 기억 The Memories Encountered - 박경태展 :: Photography',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26736'),
 ('201808',
  '윤석남展 :: Various',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26781'),
 ('201808',
  '산과함께 73 - 길 위의 리얼리즘 :: Various',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26785'),
 ('201808',
  '묵가희소·墨家喜笑展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26798'),
 ('201808',
  '나, 당신, 우리, 여기에 - 곽윤수展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26877'),
 ('201808',
  '미안한데 너무 슬퍼서 말해 줄 수 없어요 - 이준용展 :: Drawing',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26782'),
 ('201808',
  '永遠回歸 - 문수만展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26800'),
 ('201808',


html 태그가 매우 드럽게 달려있다

부적절한 tree 구조인데다가 identification 규칙이 일정치 않음, 일일이 찾기힘듦

In [277]:
artists = [t3.text for t3 in t3.find('td',{'width':'339'}).find_all('a')]
date = [el for el in t3.find('span',{'class':'red_point'}).parent]
time = [el.text for el in t3.find_all('span',{'class':'red_point'})[1:]]

전시정보를 가져오는 함수 get_info를 짰는데 next.next.next... 이런식으로 드럽다, 어쩔수없었다

In [278]:
def get_info(url):
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    t3 = soup.select('table')[3]

    info = []
    for el in [el.next.next.next.next.next for el in t3.find_all('td',{'width':'84','height':'18','align':'left'})]: #전시정보 뽑아오기, tag가 드러움
        if el.string==None: #class 'bs4.element.Tag'
            info.append(el.text)
        else:                          #class 'bs4.element.NavigableString'
            info.append(el.string) 

    artists = list(map(lambda s:s.strip(), info[0].split(',')))
    articles = [td.next['href'] for td in t3.find_all('td',{'width':'510'})[1:]]

    del info[0]
    info.insert(0,artists)
    info.append(articles)
    
    return info

In [231]:
get_info('http://www.arthub.co.kr/sub01/board03_view.htm?No=26567')

[['강지만',
  '김경섭',
  '김용석',
  '김아영',
  '문기전',
  '박정원',
  '박정용',
  '박흥선',
  '성태진',
  '송지연',
  '윤기원',
  '이수정',
  '이재열',
  '진선미'],
 '2018-08-01 ▶ 2018-08-04',
 '없음',
 'Open 14:00 ~ Close 20:00',
 '아트팩토리 후(ART FACTORY hoo)\xa0 다른전시 보기',
 '강원도 원주시 문막읍 비야동길 10-12 (후용리 1318) 나동창고 ',
 '010-4014-7488',
 'www.facebook.com/artfactoryhoo',
 ['http://www.newsis.com/view/?id=NISX20180710_0000359747&cid=10701']]

텍스트를 가져오는 함수 get_text

In [280]:
def get_text(url):
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    
    ps = [p.text for p in soup.find_all('p',{'align':'justify'})]
    text = ''.join(ps)
    
    return text.strip()

In [238]:
get_text('http://www.arthub.co.kr/sub01/board03_view.htm?No=19282')

'The birth of Life\n        물은 모든 생명의 원천이다. 모든 사물은 물에서 시작되어 그 끝 역시 물로 마무리 되고 그 과정은 지속적인 순환과정을 겪는다. \n        그리고 물의 순환과정을 통해 우리는 생명의 강인함과 진리를 깨닫게 된다. 또한 물에서 비롯된 생명은 하나가 모여 전체가 되듯이 \n        한 화면 속에서 개개의 소재들이 모여 하나의 화면을 구성하며 전일적인 개체로 성장한다.\n\n\n        The song of Life\n        모든 생명은 아름답고 작가의 작업은 그 아름다움을 노래하기 위한 수단이다. 그리고 그 아름다움을 만들어내는 모든 생명의 기원은 \n        물이다. 그래서 그의 작품 속에는 물을 상징하는 소재들이 그려져 있다. 또한 물위에서 기원하듯 그 주위에는 생명력을 품은 아름다운 \n        꽃들이 둘러싸며 저마다의 생명력을 뿜어내고 있다.The \n        energy of Life\n        생명은 항상 강인함을 지닌다. 어떠한 시련에도 생명의 순환은 결코 멈추지 않는다. 그리고 이것은 생명의 원천인 물이 가지는 에너지이기도 \n        하다. 작가는 이러한 끊임없는 순환의 과정 속의 생명이 가지는 강인함과 아름다움을 노래하기 위해 캔버스 위를 유화와 아크릴을 \n        통해 다채로운 색채로 표현한다. 잔잔한 한 폭의 정물화 인듯하지만 화려하고 과감한 색채의 대비는 우리에게 생명의 에너지를 속삭인다. \n         ⓒ'

샘플 데이터를 만들어보자, 55개의 전시만 가져왔다

In [242]:
arts = []
for y in range(2018,2019): # 년: 2018
    for m in range(1,2): # 월:1
        if m<10:
            m = '0'+str(m)
        else:
            m = str(m)
        for p in range(1,2): # page: 1
            try:
                links = get_links(y,m,p)
                for link in links:
                    info = get_info(link[2])
                    text = get_text(link[2])
                    
                    temp = [*link,*info,text]
                    
                    arts.append(temp)

            except Exception as e:
                print(e)
                continue

data = pd.DataFrame(arts,columns=['연월','전시제목','링크','전시작가','전시일정','초대일시','관람시간','전시장소','주소','연락처','홈페이지','아티클','텍스트'])
data.sample(10)

,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트
13,201801,2018 Cre8tive Report :: Various,http://www.arthub.co.kr/sub01/board03_view.htm...,"[김푸르나, 나광호, 라오미, 박형진, 사윤택, 이현호, 허수영, 周代焌]",2018-01-11 ▶ 2018-03-03,2018-01-11 PM 5:00,"Open 10:00 ~ Close 18:00(수~21:00, 일월 휴관)",OCI미술관(OCI Museum of Art) 다른전시 보기,서울시 종로구 우정국로 45-14,02-734-0440-1,www.ocimuseum.org,[http://www.economytalk.kr/news/articleView.ht...,"OCI미술관 창작스튜디오 7기 입주 작가 일곱의 한 해 성과를 돌아보는 전시, 《2..."
32,201801,Universe-Falling in Blue '18 - 박종철展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,[박종철(Park Jongchul 朴鍾喆)],2018-01-19 ▶ 2018-02-03,2018-01-19 AM 12:00,Open 10:00 ~ Close 18:00(월요일 휴관),겸재정선미술관 다른전시 보기,서울시 강서구 가양1동 243-1,02-2659-2206,www.gjjs.or.kr,[],겸재정선미술관에서는 ‘2018 겸재정선미술관 유수작가초청기획전시’의 일환으로 박종철...
26,201801,Good Life - 김원展 :: Painting & Drawing,http://www.arthub.co.kr/sub01/board03_view.htm...,[김원(Kim Won 金元)],2018-01-17 ▶ 2018-02-14,,Open 10:00 ~ Close 18:00,갤러리 밈(GALLERY MEME) 다른전시 보기,서울시 종로구 인사동 5길 3,02-733-8877,www.gallerymeme.com,[http://www.jjan.kr/news/articleView.html?idxn...,● Good Life\n 서로의 마음을 알아가는 방법에 있어 나는 좋은...
49,201801,The Day in the Evening 데이 인 이브닝 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,"[피비 언윈 Phoebe Unwin, 톰 워스폴드 Tom Worsfold, 이진한]",2018-01-29 ▶ 2018-03-09,없음,Open 10:00 ~ Close 18:00(일요일 휴관),스페이스K(space K) 과천 다른전시 보기,경기도 과천시 별양동 1-23 코오롱 타워 1F,02-3677-3105,www.spacek.co.kr,[http://www.edaily.co.kr/news/news_detail.asp?...,코오롱의 문화예술 나눔공간 스페이스K에서는 2018년 새해 첫 전시로 런던의 신진작...
44,201801,Bouquet for Someone - 성민우展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,[성민우(Sung Minwoo 成民友)],2018-01-25 ▶ 2018-02-07,2018-01-25 AM 11:00,Open 10:00 ~ Close 18:00(월요일 휴관),모리스갤러리(Morris Gallery) 다른전시 보기,대전시 유성구 도룡동 397-1,042-867-7009,www.morrisgallery.co.kr,[http://www.daejonilbo.com/news/newsitem.asp?p...,● Bouquet for Someone\n 오이코스로 그려진 풀숲에 서...
14,201801,My Wonderful Dream - 이서미展 :: Various,http://www.arthub.co.kr/sub01/board03_view.htm...,[이서미(Lee Seomi 李瑞薇)],2018-01-11 ▶ 2018-02-04,없음,"Open 10:30 ~ Close 20:00(금토일 ~20:30, 백화점 휴점일 휴관)",롯데갤러리 잠실점 다른전시 보기,서울시 송파구 잠실동 40-1 롯데백화점 12F,02-411-6911,,[],"어린 시절을 기억하는 것과 어제 밤에 꾼 꿈을 기억하는 것, 보았던 것이나 들었던 ..."
24,201801,산색山色 - 소원섭展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,[소원섭(So Wonsub 蘇元燮)],2018-01-17 ▶ 2018-01-30,없음,Open 11:00 ~ Close 19:00,갤러리 마롱(GALLERY MARRON) 다른전시 보기,서울시 종로구 북촌로 143-6,02-720-4540,www.gallerymarron.com,[],
22,201801,이은희展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,[이은희(Lee Eunhee 李銀姬)],2018-01-16 ▶ 2018-02-12,없음,Open 0:00 ~ Close 24:00,아트허브 온라인 갤러리(ARTHUB Online Gallery) 다른전시 보기,온라인 스페이스(Online Space),arthub2@naver.com,www.arthub.co.kr,[],● 이은희展\n 최근 제작하고 있는 주제들은 대중적이며 가벼운 일상 이...
4,201801,Vanishing Point - 이강훈展 :: Conceptual Art,http://www.arthub.co.kr/sub01/board03_view.htm...,[이강훈(Lee Kanghoon 李康熏)],2018-01-05 ▶ 2018-01-17,없음,Open 11:00 ~ Close 18:00(월요일 휴관),예술공간 서:로 다른전시 보기,서울시 용산구 신흥로 36길 6,seoro-art@naver.com,blog.naver.com/seoro-art,[],● Vanishing Point\n 관계성은 인간의 본질이다. 자아는 ...
25,201801,그곳…마주하다 There…Lay gentle eyes on... - 김성남展 :: ...,http://www.arthub.co.kr/sub01/board03_view.htm...,[김성남(Kim Sungnam 金成男)],2018-01-17 ▶ 2018-02-12,2018-01-17 PM 5:00,Open 10:00 ~ Close 19:00(일요일 휴관),금산갤러리(KEUMSAN GALLERY) 다른전시 보기,서울시 중구 소공로 46 쌍용남산플래티넘 B-103,02-3789-6317,www.keumsan.org,[http://news.mk.co.kr/newsRead.php?year=2018&n...,금산갤러리에서는 김성남 작가의 ‘그곳… 마주하다’라는 타이틀의 개인전을 개최한다. ...


In [243]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 13 columns):
연월      55 non-null object
전시제목    55 non-null object
링크      55 non-null object
전시작가    55 non-null object
전시일정    55 non-null object
초대일시    55 non-null object
관람시간    55 non-null object
전시장소    55 non-null object
주소      55 non-null object
연락처     55 non-null object
홈페이지    55 non-null object
아티클     55 non-null object
텍스트     55 non-null object
dtypes: object(13)
memory usage: 5.7+ KB


In [246]:
corpus = ''
for txt in data.텍스트:
    corpus +=txt

In [247]:
corpus #텍스트를 합쳐 코퍼스를 만든다

'● 천 년의 꽃 Millennium Flowers\n        전시되는 “천 년의 꽃”은 불교의 불상이 놓여진 단상을 수미단(須彌壇), 영기단(靈氣壇)이라 하는데 이 표면에 그려지고 조각된 \n        영기꽃(연화, 목단, 국화)을 통해 절대적 진리를 상징하는 작가 자신의 심상을 꽃으로 형상화하고 검 프린트(Gum Bichromate \n        Print)기법으로 채색을 하여 제작된 작품이다. 작가는 작품을 구상하고 구현하기까지 많은 워크플로우(Workflow)를 거치면서, \n        심리적이며 주관적 시간으로 재발견(rediscovery) 된 결과물이 천년의 꽃(Millennium Flowers)으로 시각화되어 \n        발표하게 되었다. \n\n\n        불교문화유물(수미단)에 담겨진 꽃은 만물이 생성하는 도상이며, 정면을 향하고 있고 형태가 겹쳐지는 부분에 투각의 방법에 의하여 \n        선의 두께감이 은유적으로 표현되어있다. 이는 현대 디자인의 픽토그램(pictogram)과 유사성이 있다. 이것은 종교적 치유와 \n        염원이 강조되고 단순화된 표현이 불교미술이나 티벳의 모래로 만든 만다라, 인디언의 모래그림에서 볼 수 있다. 서양 미술에서는 \n        브루넬레스키(Filippo Brunelleschi)의 원근법을 적용하기 전, 지오토 디 본도네(Giotto di Bondone)는 \n        그림을 단순화 시키고 깊이감을 없애려고 하였다. 이후 에두아르 마네(Edouard Manet)에 의하여 실험적인 평면성과 환영주의를 \n        거부하는 그림이 표현된다.\n\n"천 년의 꽃" \n        연작은 원근감을 의도적으로 단순화 하고 객관성을 갖게 하려고 사실주의적 복원과는 거리를 둔 컨템포러리 아트(contemporary \n        art)로 재구성 하려고 시도하였다. 그럼에도 촬영된 결과는 부감 촬영된 앵글처럼 느껴지거나 스캐너로 압축된 것처럼 보여 진다. \n  

아래부터는 텍스트 전처리 과정

In [260]:
import nltk
from konlpy.tag import Kkma,Twitter
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize
from matplotlib import font_manager, rc

In [255]:
kkma = Kkma()
# twitter = Twitter()

In [266]:
tokens = kkma.morphs(corpus)

In [282]:
word_tokenize(corpus)[:15]

['●',
 '천',
 '년의',
 '꽃',
 'Millennium',
 'Flowers',
 '전시되는',
 '“',
 '천',
 '년의',
 '꽃',
 '”',
 '은',
 '불교의',
 '불상이']

In [268]:
count2 = Counter(tokens)
count2.most_common(30) #불용어 처리를 하지않은 결과이다

[('하', 2316),
 ('.', 2108),
 ('ㄴ', 2062),
 ('이', 1936),
 ('의', 1896),
 ('는', 1877),
 ('을', 1493),
 (',', 1413),
 ('다', 1025),
 ('은', 986),
 ('에', 860),
 ('어', 802),
 ('고', 743),
 ('를', 732),
 ('ㄴ다', 645),
 ('있', 612),
 ('되', 608),
 ('과', 555),
 ('으로', 482),
 ('들', 474),
 ('에서', 457),
 ('가', 450),
 ('the', 426),
 ('것', 406),
 ('ㄹ', 395),
 ('로', 341),
 ('’', 330),
 (')', 325),
 ('작가', 314),
 ('게', 311)]

In [269]:
text = nltk.Text(tokens)

In [273]:
text.similar("작가")

그 우리 하나 회화 작품 나 이 작업 형태 의미 인간 시간 이미지 예술가 사람 존재 자연 나무 삶 색


In [274]:
text.similar("존재")

이미지 작가 회화 작업 그 주제 작품 다양 공간 사람 요소 세계 장소 이야기 자연 상태 경계 단계 차이 시간


In [276]:
text.similar("희망")

것 시간 공간 의미 인식 개념 모습 중심 사랑 상상 년 상징 자신 작품 은유 표현 치유 그림 촬영 방식
